In [1]:
from dfply import *
import pandas as pd
import os
os.chdir('C:\\Users\\wzh55\\Desktop\\GitHub\\Guzman-Energy\\data')
data1 = pd.read_csv("Assignment 2 - USA_AL_Auburn-Opelika.AP.722284_TMY3_BASE.csv")
data2 = pd.read_csv("Assignment 2 - new.app4.csv")

data1_copy = data1.copy()
data2_copy = data2.copy()

#the main idea is that I split data/time from both files into strings, then convert them into datetime.
#Then, I group data in "Assignment 2 - new.app4.csv" by hour. Since datetime in file 1 is by hour already, if the index(date)
#are equal, merge the value.

def splittimedata1(time):  
    firstsplit = row[time].split()
    month, date = firstsplit[0].split("/")
    hour, minute, second = firstsplit[1].split(":")
    if hour == "24":
        hour = "00"
    return "2013" + month + date + hour + minute

def splittimedata2(time):
    firstsplit = row[time].split()
    month, date, year = firstsplit[0].split("/")
    hour, minute = firstsplit[1].split(":")
    if len(month) == 1:
        month = "0" + month
    if len(date) == 1:
        date = "0" + date
    return year + month + date + hour + minute 

for index, row in data2_copy.iterrows():
    data2_copy.at[index,"time"] = splittimedata2("time")   
    
for index, row in data1_copy.iterrows():
    data1_copy.at[index,"Date/Time"] = splittimedata1("Date/Time")

data1_timestamp = pd.to_datetime(data1_copy["Date/Time"], format='%Y%m%d%H%M', errors = "ignore")
data1_watts = [i/1 for i in data1_copy["Appl:InteriorEquipment:Electricity [kW](Hourly)"]]
data1_final = pd.DataFrame(index=data1_timestamp,data=data1_watts,columns=["value1"])

data2_timestamp = pd.to_datetime(data2_copy["time"], format='%Y%m%d%H%M', errors='ignore')
data2_watts = [i/1000 for i in data2_copy["W_min"]]
data2_df = pd.DataFrame(index=data2_timestamp,data=data2_watts,columns=["value2"])
data2_final = data2_df.resample('H').sum()

for index1, row1 in data1_final.iterrows():
    for index2, row2 in data2_final.iterrows():
        if index1 == index2:
            sum_value = row1["value1"] + row2["value2"]
            data1_final.at[index1, "value1"] = sum_value

data1["Appl:InteriorEquipment:Electricity [kW](Hourly)"] = [i/1 for i in data1_final["value1"]]
data_eletricity_only = data1 >> select(contains("Electricity"))
Electricity_sum = data_eletricity_only.sum(axis=1)
final_data = data1 >> mutate(Electricity_sum=Electricity_sum)

In [2]:
final_data #the sum of all electricity is on the far right

,Date/Time,Electricity:Facility [kW](Hourly),Gas:Facility [kW](Hourly),Heating:Electricity [kW](Hourly),Heating:Gas [kW](Hourly),Cooling:Electricity [kW](Hourly),HVACFan:Fans:Electricity [kW](Hourly),Electricity:HVAC [kW](Hourly),Fans:Electricity [kW](Hourly),General:InteriorLights:Electricity [kW](Hourly),General:ExteriorLights:Electricity [kW](Hourly),Appl:InteriorEquipment:Electricity [kW](Hourly),Misc:InteriorEquipment:Electricity [kW](Hourly),Water Heater:WaterSystems:Electricity [kW](Hourly),Electricity_sum
0,01/01 01:00:00,0.974334,4.452977,0.0,4.425010,0.0,0.112709,0.112709,0.112709,0.154019,0.033180,0.092943,0.406035,0.158803,2.157443
1,01/01 02:00:00,0.796582,4.850317,0.0,4.824566,0.0,0.122617,0.122617,0.122617,0.089845,0.019355,0.076186,0.373851,0.098084,1.821755
2,01/01 03:00:00,0.735028,5.037645,0.0,5.012193,0.0,0.127099,0.127099,0.127099,0.064175,0.013825,0.062326,0.369517,0.081442,1.707610
3,01/01 04:00:00,0.727433,5.107562,0.0,5.082468,0.0,0.128391,0.128391,0.128391,0.064175,0.013825,0.053976,0.364315,0.086107,1.695004
4,01/01 05:00:00,0.778706,5.270878,0.0,5.246732,0.0,0.132549,0.132549,0.132549,0.064175,0.013825,0.065823,0.350553,0.135137,1.805868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12/31 20:00:00,2.601121,0.044507,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.743542,0.160179,0.349591,0.646167,0.684999,5.185598
8756,12/31 21:00:00,2.445630,0.046038,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.677155,0.145877,0.310730,0.668381,0.626843,4.874615
8757,12/31 22:00:00,2.206391,0.044963,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.570934,0.122994,0.293949,0.652777,0.549092,4.396137
8758,12/31 23:00:00,1.769166,0.295330,0.0,0.256420,0.0,0.006642,0.006642,0.006642,0.424881,0.091531,0.234174,0.564895,0.430399,3.534973
